In [14]:
from google.colab import drive
drive.mount('/content/drive')


ModuleNotFoundError: No module named 'google.colab'

In [ ]:
!git clone https: // github.com/namnv3899/NLP.git


Cloning into 'NLP'...
remote: Enumerating objects: 63, done.
remote: Counting objects: 100% (63/63), done.
remote: Compressing objects: 100% (50/50), done.
remote: Total 63 (delta 10), reused 60 (delta 9), pack-reused 0
Unpacking objects: 100% (63/63), done.


In [ ]:
!pip install vncorenlp transformers fairseq seaborn sklearn wordcloud


     |████████████████████████████████| 2.6 MB 8.3 MB/s 
     |████████████████████████████████| 3.4 MB 71.1 MB/s 
     |████████████████████████████████| 1.7 MB 62.9 MB/s 
     |████████████████████████████████| 3.3 MB 76.0 MB/s 
     |████████████████████████████████| 895 kB 58.3 MB/s 
     |████████████████████████████████| 67 kB 5.5 MB/s 
     |████████████████████████████████| 596 kB 55.4 MB/s 
     |████████████████████████████████| 90 kB 13.0 MB/s 
     |████████████████████████████████| 145 kB 84.0 MB/s 
     |████████████████████████████████| 112 kB 78.1 MB/s 
     |████████████████████████████████| 74 kB 4.3 MB/s 
  Created wheel for vncorenlp: filename=vncorenlp-1.0.3-py3-none-any.whl size=2645951 sha256=417c9642692abe8086e284e290af5c94bc8347307a0e352971342add83ce771c
  Stored in directory: /root/.cache/pip/wheels/0c/d8/f2/d28d97379b4f6479bf51247c8dfd57fa00932fa7a74b6aab29
  Created wheel for antlr4-python3-runtime: filename=antlr4_python3_runtime-4.8-py3-none-any.whl size=1

In [ ]:
!git clone https: // huggingface.co/vinai/phobert-base


Cloning into 'phobert-base'...
remote: Enumerating objects: 36, done.
remote: Counting objects: 100% (36/36), done.
remote: Compressing objects: 100% (35/35), done.
remote: Total 36 (delta 14), reused 0 (delta 0)
Unpacking objects: 100% (36/36), done.


In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud
import pickle
from utils import load_yaml, timing
from nlp import NLP
from training import *

from tqdm.notebook import tqdm

tqdm.pandas(desc='Processing Dataframe')


2022-01-19 09:40:26.735790: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0


In [2]:
PATHS = load_yaml('paths')
PHOBERT_DATA = PATHS['phobert_base']


# Prepare data and preprocess

In [ ]:
df = load_data()
df.info()


In [ ]:
df['category'].unique()


In [ ]:
def preprocess_data(raw_df, stored_list=None):
    raw_df = raw_df.copy()

    # drop null value
    raw_df.drop(raw_df[(raw_df.category.isnull())
                       | raw_df.text.isnull()].index,
                axis=0,
                inplace=True)

    # preprocess text
    raw_df['text'] = raw_df['text'].progress_apply(NLP().preprocess_text)

    # drop rows that have len(clean_body) < 10
    len_texts = raw_df['text'].astype(str).apply(lambda t: len(t.split()))
    raw_df.drop(len_texts[len_texts <= 10].index, axis=0, inplace=True)

    print(f'preprocess data complete! processed data shape: {raw_df.shape}')
    if stored_list is not None:
        stored_list.append(raw_df)

    return raw_df


def multi_preprocess_data(raw_df, n_cores=4):
    manager = multiprocessing.Manager()
    dfs = manager.list()
    jobs = []
    offset = int(raw_df.shape[0] / n_cores)
    start_index = 0

    for i in range(n_cores):
        if i == n_cores - 1:
            part_df = raw_df.iloc[start_index:, :]
        else:
            part_df = raw_df.iloc[start_index:start_index + offset, :]

        start_index += offset

        p = multiprocessing.Process(target=preprocess_data,
                                    args=(part_df, dfs))
        jobs.append(p)
        print(f'Start process {i}')
        p.start()

    for p in jobs:
        p.join()

    return pd.concat(dfs, axis=0).reset_index(drop=True)


In [ ]:
clean_df = multi_preprocess_data(df, 6)
clean_df.info()


In [ ]:
clean_df.to_csv('data/data2_keep_stopwords.csv', index=False)
clean_df


In [3]:
clean_df = pd.read_csv('data/raw_data.csv')
clean_df


,id,category,text
0,1,thể thao,một granada xếp áp_chót bảng tất_nhiên không_t...
1,2,thể thao,ở trận này hlv pep guardiola tung ra sân đội_h...
2,3,thể thao,ở trận này tomas berdych đã chơi không tốt vì ...
3,4,thể thao,làm thế_nào để có_thể kéo khán_giả đến sân cđv...
4,5,thể thao,môn quần_vợt không đòi_hỏi sức_mạnh quá mức nê...
...,...,...,...
141291,155589,kinh doanh,khách_hàng có_thể đặt lịch bảo_dưỡng sửa_chữa ...
141292,155613,kinh doanh,nhà_sáng_lập tesla đã khởi_động năm 2022 với m...
141293,155622,kinh doanh,được phát_triển bởi vinbigdata một công_ty côn...
141294,155623,kinh doanh,các bác tài cho biết rất nóng_lòng được tham_g...


In [5]:
len(clean_df['category'].unique())


9

In [ ]:
category_count = clean_df.groupby('category')['text'].count().sort_values(
    ascending=False)
print(category_count)
plt.figure(figsize=(18, 12))
sns.barplot(x=category_count.index, y=category_count)
plt.xticks(rotation=90)
plt.show()


In [ ]:
lentext_count = clean_df['text'].apply(lambda x: len(x.split()))
lentext_count.describe()


In [ ]:
sns.boxplot(lentext_count)


In [ ]:
plt.figure(figsize=(15, 30))
wc = WordCloud(max_words=100, width=800, height=600)

for idx, category in enumerate(labels):
    plt.subplot(5, 2, idx + 1)
    wc.generate(" ".join(clean_df.loc[clean_df.category == category, 'text']))
    plt.imshow(wc, interpolation='bilinear')
    plt.title(category)
    plt.axis("off")
plt.figure()


# Training

In [ ]:
from training import *


### SVM

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    clean_df['text'],
    clean_df['category'],
    test_size=0.2,
    stratify=clean_df['category'])
print(f'X_train shape: {X_train.shape}')
print(f'X_test shape:  {X_test.shape}')
print(f'y_train shape:  {y_train.shape}')
print(f'y_test shape:  {y_test.shape}')


X_train shape: (110593,)
X_test shape:  (27649,)
y_train shape:  (110593,)
y_test shape:  (27649,)


In [ ]:
clf = svm_straing(X_train, y_train)


In [ ]:
from sklearn.metrics import classification_report

y_pred = clf.predict(X_test)
with open('data/classification_report.txt', 'a') as f:
    report = classification_report(y_test, y_pred)
    f.write(report)
    f.write('=' * 53 + '\n')


In [3]:
with open('data/model.pickle', 'rb') as f:
    clf = pickle.load(f)


In [10]:
with open('data/test.txt', 'r', ) as f:
    text = f.read()
text


'Theo BS Trương Hữu Khanh, Cố vấn chuyên môn Khoa Nhiễm Khuẩn - thần kinh, Bệnh viện Nhi đồng 1, TP.HCM, xuất hiện các triệu chứng mệt mỏi, khó thở, đau đầu…sau khi khỏi Covid-19, không phải đều là do hậu Covid-19. Chỉ một số người thực sự mắc di chứng sau khi khỏi bệnh, còn lại là dấu hiệu của các bệnh lý khác nhưng bị nhầm lẫn thành bất thường hậu Covid-19.\n\n“Một số dấu hiệu không phải hậu Covid-19 mà là bệnh khác. Ví dụ có trường hợp trẻ sốt, người nhà đinh ninh là hậu Covid nhưng thực ra lại là sốt xuất huyết.\n\nNgười khó thở cũng bị suy luận là hậu Covid nhưng thực chất lại là hen suyễn. Có trường hợp ho nhiều, họ tưởng nhầm là di chứng của Covid nhưng đi khám mới phát hiện ra bị bệnh lao. Vì vậy chúng ta phải lưu ý, phòng trường hợp chỉ chăm chăm lo hậu Covid, bỏ sót các bệnh khác, làm bệnh nặng thêm”, bác sĩ Khanh nhấn mạnh'

In [11]:
clf.predict([text])


array(['sức khỏe'], dtype=object)

### Bert

In [3]:
from phobert.train_classifer_keras import load_text, PhoBertTraining
from phobert.embeding import Embeding
import logging
logging.disable(logging.WARNING)


2022-01-18 02:53:19.206713: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0


In [ ]:
ids_train = load_text(X_train.tolist())
ids_test = load_text(X_test.tolist())

print(len(ids_train))
print(len(y_train))
print(len(ids_test))
print(len(y_test))

with open("ids_train", 'wb') as f:
    pickle.dump(ids_train, f)
with open("y_train", 'wb') as f:
    pickle.dump(y_train, f)

with open("ids_test", 'wb') as f:
    pickle.dump(ids_test, f)
with open("y_test", 'wb') as f:
    pickle.dump(y_test, f)

print("LOAD DATA DONE")


In [4]:
with open("data/ids_train", "rb") as f:
    ids_train = pickle.load(f)
with open("data/y_train", "rb") as f:
    y_train = pickle.load(f)
with open("data/ids_test", "rb") as f:
    ids_test = pickle.load(f)
with open("data/y_test", "rb") as f:
    y_test = pickle.load(f)

with open('data/label_encoder.pickle', 'rb') as f:
    le = pickle.load(f)

print(ids_train)
print(le.classes_)
print("LOAD DATA FROM FILE DONE")


[[   0  405 1058 ...    1    1    2]
 [   0   63  200 ... 8141 4481    2]
 [   0 5689 5301 ... 4104 2158    2]
 ...
 [   0   78  210 ...   11  320    2]
 [   0  227  400 ...   57 8677    2]
 [   0   63  538 ...    6  200    2]]
['giáo dục' 'giải trí' 'kinh doanh' 'pháp luật' 'sức khỏe' 'thế giới'
 'thể thao' 'thời sự' 'văn hóa']
LOAD DATA FROM FILE DONE


In [5]:
print(ids_train.shape)
print(ids_test.shape)
print(y_train.shape)
print(y_test.shape)


(113036, 256)
(28260, 256)
(113036,)
(28260,)


In [6]:
y_train_encoded = le.transform(y_train)
y_test_encoded = le.transform(y_test)

emb = Embeding(PHOBERT_DATA)

training = PhoBertTraining(ids_train,
                           y_train_encoded,
                           ids_test,
                           y_test_encoded,
                           le.classes_)
training.make_generator(emb.get_emb_vector, batch_size=512)
training.build_model()
history = training.fit(n_epochs=10)
training.plot_train_history(history)


type of all dataset
<class 'numpy.ndarray'> <class 'numpy.ndarray'> <class 'numpy.ndarray'> <class 'numpy.ndarray'>
==> Make data generator
==> Make data generator success
train_generator: 220
valid_generator: 55
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d (Conv1D)              (None, 252, 128)          491648    
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 250, 64)           24640     
_________________________________________________________________
global_max_pooling1d (Global (None, 64)                0         
_________________________________________________________________
dense (Dense)                (None, 32)                2080      
_________________________________________________________________
dense_1 (Dense)              (None, 9)                 297       
Total params: 518,665
Trainable params: 5

2022-01-18 02:53:39.571419: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2022-01-18 02:53:39.571549: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2022-01-18 02:53:39.571670: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:941] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-01-18 02:53:39.571925: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:02:00.0 name: GeForce MX130 computeCapability: 5.0
coreClock: 1.189GHz coreCount: 3 deviceMemorySize: 1.96GiB deviceMemoryBandwidth: 37.33GiB/s
2022-01-18 02:53:39.571946: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
2022-01-18 02:53:39.590787: I tensorflow/stream_executor/platform/default/dso_lo

Epoch 1/10


## datasets

In [2]:
from training import PhobertTraining


In [3]:
training = PhobertTraining('data/phobert-base')


In [4]:
with open('data/training_data/label_encoder.pickle', 'rb') as f:
    label_encoder = pickle.load(f)
training.make_data('data/raw_data.csv',
                   label_encoder=label_encoder,
                   test_size=0.2,
                   batch_size=2)


Using custom data configuration default-31a95d250e5386b0
Reusing dataset csv (/home/long/.cache/huggingface/datasets/csv/default-31a95d250e5386b0/0.0.0/6b9057d9e23d9d8a2f05b985917a0da84d70c5dae3d22ddd8a3f22fb01c69d9e)
100%|██████████| 1/1 [00:00<00:00, 19.21it/s]
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Loading cached processed dataset at /home/long/.cache/huggingface/datasets/csv/default-31a95d250e5386b0/0.0.0/6b9057d9e23d9d8a2f05b985917a0da84d70c5dae3d22ddd8a3f22fb01c69d9e/cache-92a22ed49c5f65ac.arrow
Loading cached split indices for dataset at /home/long/.cache/huggingface/datasets/csv/default-31a95d250e5386b0/0.0.0/6b9057d9e23d9d8a2f05b985917a0da84d70c5dae3d22ddd8a3f22fb01c69d9e/cache-83c29265ed322875.arrow and /home/long/.cache/huggingface/datasets/csv/default-31a95d250e5386b0/0.0.0/6b9057d9e23d9d8a2f05b985917a0da84d70c5dae3d22ddd8a3f22fb01c69d9e/cache-17c01223bc6acbb1.arrow
2022-01-19 09:40:58.907035: I 

train dataset shape: <TensorSliceDataset shapes: ({input_ids: (512,), token_type_ids: (512,), attention_mask: (512,)}, ()), types: ({input_ids: tf.int64, token_type_ids: tf.int64, attention_mask: tf.int64}, tf.int64)>
validation dataset shape: <TensorSliceDataset shapes: ({input_ids: (512,), token_type_ids: (512,), attention_mask: (512,)}, ()), types: ({input_ids: tf.int64, token_type_ids: tf.int64, attention_mask: tf.int64}, tf.int64)>


In [5]:
training.fit()

2022-01-19 09:41:14.724720: I tensorflow/core/profiler/lib/profiler_session.cc:136] Profiler session initializing.
2022-01-19 09:41:14.724751: I tensorflow/core/profiler/lib/profiler_session.cc:155] Profiler session started.
2022-01-19 09:41:14.725998: I tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1365] Profiler found 1 GPUs
2022-01-19 09:41:14.736743: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'libcupti.so.11.0'; dlerror: libcupti.so.11.0: cannot open shared object file: No such file or directory
2022-01-19 09:41:14.736917: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'libcupti.so'; dlerror: libcupti.so: cannot open shared object file: No such file or directory
2022-01-19 09:41:14.736939: E tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1415] function cupti_interface_->Subscribe( &subscriber_, (CUpti_CallbackFunc)ApiCallback, this)failed with error CUPTI could not be 

init model successfully!
Model: "tf_roberta_for_sequence_classification"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
roberta (TFRobertaMainLayer) multiple                  134407680 
_________________________________________________________________
classifier (TFRobertaClassif multiple                  597513    
Total params: 135,005,193
Trainable params: 597,513
Non-trainable params: 134,407,680
_________________________________________________________________
None
Epoch 1/10


2022-01-19 09:41:30.391536: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
2022-01-19 09:41:30.489728: I tensorflow/core/platform/profile_utils/cpu_utils.cc:112] CPU Frequency: 1800000000 Hz


InvalidArgumentError:  indices[0,480] = 482 is not in [0, 258)
	 [[node tf_roberta_for_sequence_classification/roberta/embeddings/Gather_1 (defined at home/long/CodeWorkspace/NLP/venv/lib/python3.8/site-packages/transformers/models/roberta/modeling_tf_roberta.py:167) ]] [Op:__inference_train_function_24407]

Errors may have originated from an input operation.
Input Source operations connected to node tf_roberta_for_sequence_classification/roberta/embeddings/Gather_1:
 tf_roberta_for_sequence_classification/roberta/embeddings/add_1 (defined at home/long/CodeWorkspace/NLP/venv/lib/python3.8/site-packages/transformers/models/roberta/modeling_tf_roberta.py:129)

Function call stack:
train_function
